![](https://user-images.githubusercontent.com/83436724/166116197-cbe1a8bc-e1df-4cac-ad5e-9f6cd9f0f43c.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.cloud import bigquery
import os
from google.oauth2 import service_account
os.chdir('..')
warnings.filterwarnings('ignore')

In [5]:
#### Traininig step


service_account_path = "bq_key.json"

# BigQuery client
client = bigquery.Client.from_service_account_json(service_account_path)

# SQL Query
query = """
SELECT *
FROM `proyectodata-348005.precios.scrapped_data` 
"""

dataframe = client.query(query).to_dataframe()

# Dataframe
dataframe.head()

,title,seller,offer_price,regular_price,catalog_price,brand,image
0,"Macbook Pro 2015 15"" A1309 Intel Core I7 16gb ...",Por Unclic,1590.0,1900.0,NaN,APPLE,https://www.falabella.com.pe/cdn-cgi/imagedeli...
1,Macbook Pro 16 M3 Max 48gb Ram + 1tb Ssd Tecla...,Por Wetech,20459.0,23000.0,NaN,APPLE,https://www.falabella.com.pe/cdn-cgi/imagedeli...
2,Macbook Air 15 M3 8gb Ram + 256gb Ssd Teclado ...,Por Wetech,7459.0,8000.0,NaN,APPLE,https://www.falabella.com.pe/cdn-cgi/imagedeli...
3,Macbook Pro 14 M3 Pro 18gb Ram + 1tb Ssd Tecla...,Por Wetech,11999.0,13200.0,NaN,APPLE,https://www.falabella.com.pe/cdn-cgi/imagedeli...
4,Macbook Pro 16 M3 Max 48gb Ram + 1tb Ssd Tecla...,Por Wetech,19959.0,22000.0,NaN,APPLE,https://www.falabella.com.pe/cdn-cgi/imagedeli...


In [21]:
def clean_title(text):
    text = text.lower()
    text = text.replace('"', '')
    return text

dataframe['title'] = dataframe['title'].apply(clean_title)
dataframe = dataframe[~dataframe['offer_price'].isna()]

In [22]:
### Train Test Split

from sklearn.model_selection import train_test_split

X = dataframe['title']
y = dataframe['offer_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer for parsing/counting words
vectorizer = CountVectorizer()
x_train_vect = vectorizer.fit_transform(X_train)

### Transforming the data

x_test_vect = vectorizer.transform(X_test)

In [24]:
### Training the model

from sklearn.naive_bayes import MultinomialNB

# Create the model

model = MultinomialNB()
model.fit(x_train_vect, y_train)

MultinomialNB()

In [35]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=123) 
model.fit(x_train_vect, y_train)

RandomForestRegressor(random_state=123)

In [37]:
from xgboost import XGBRegressor    
model = XGBRegressor(random_state=123)
model.fit(x_train_vect, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=123, ...)

In [38]:
### Analyzing the model
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
print(mean_absolute_error(y_test,model.predict(x_test_vect)))
mean_absolute_percentage_error(y_test,model.predict(x_test_vect))


310.357726456283


np.float64(0.10840025103664484)